# Batch Normalization

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from funkcije.fc_net import *
from funkcije.data_utils import get_CIFAR10_data
from funkcije.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from funkcije.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ vraća relativnu grešku """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def print_mean_std(x,axis=0):
    print('  srednja vrijednost: ', x.mean(axis=axis))
    print('  standardna devijacija:  ', x.std(axis=axis))
    print() 

In [ ]:
#Učitati CIFAR-10 podatke
data = get_CIFAR10_data()
for k, v in data.items():
    print('%s: ' % k, v.shape)

## Batch normalizacija: prolaz unaprijed
U fajlu `funkcije/layers.py`, implementirati batch normalizacija prolaz unaprijed u funkciji `batchnorm_forward`. 

In [ ]:
# Simulacija prolaza unaprijed za dvoslojnu mrežu
np.random.seed(231)
N, D1, D2, D3 = 200, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print('Prije batch normalizacije:')
print_mean_std(a,axis=0)

gamma = np.ones((D3,))
beta = np.zeros((D3,))
# Srednje vrijednosti će biti približno nuli, a standardne devijacije jedinici
print('Posle batch normalizacije (gamma=1, beta=0)')
a_norm, _ = batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=0)

gamma = np.asarray([1.0, 2.0, 3.0])
beta = np.asarray([11.0, 12.0, 13.0])
# Sada srednje vrijednosti treba da budu blizu beta, a standardne devijacije blizu gamma
print('Posle batch normalizacije (gamma=', gamma, ', beta=', beta, ')')
a_norm, _ = batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=0)

In [ ]:
np.random.seed(231)
N, D1, D2, D3 = 200, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print('Prije batch normalizacije:')
print_mean_std(a,axis=0)

gamma = np.ones((D3,))
beta = np.zeros((D3,))
# Srednje vrijednosti će biti približno nuli, a standardne devijacije jedinici
print('Posle batch normalizacije (gamma=1, beta=0)')
a_norm, _ = batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=0)

gamma = np.asarray([1.0, 2.0, 3.0])
beta = np.asarray([11.0, 12.0, 13.0])
# Sada srednje vrijednosti treba da budu blizu beta, a standardne devijacije blizu gamma
print('Posle batch normalizacije (gamma=', gamma, ', beta=', beta, ')')
a_norm, _ = batchnorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=0)

In [ ]:
np.random.seed(231)
N, D1, D2, D3 = 200, 50, 60, 3
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)

bn_param = {'mode': 'train'}
gamma = np.ones(D3)
beta = np.zeros(D3)

for t in range(50):
    X = np.random.randn(N, D1)
    a = np.maximum(0, X.dot(W1)).dot(W2)
    batchnorm_forward(a, gamma, beta, bn_param)
    

bn_param['mode'] = 'test'
X = np.random.randn(N, D1)
a = np.maximum(0, X.dot(W1)).dot(W2)
a_norm, _ = batchnorm_forward(a, gamma, beta, bn_param)

# Srednje vrijednosti će biti približno nuli, a standardne devijacije jedinici, ali će biti pod većim šumom
print('Posle batch normalizacije (test-time):')
print_mean_std(a_norm,axis=0)

## Batch normalizacija: prolaz unazad
Sada implementirajte prolaz unazad u funkciji `batchnorm_backward`.

In [ ]:
np.random.seed(231)
N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
fx = lambda x: batchnorm_forward(x, gamma, beta, bn_param)[0]
fg = lambda a: batchnorm_forward(x, a, beta, bn_param)[0]
fb = lambda b: batchnorm_forward(x, gamma, b, bn_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma.copy(), dout)
db_num = eval_numerical_gradient_array(fb, beta.copy(), dout)

_, cache = batchnorm_forward(x, gamma, beta, bn_param)
dx, dgamma, dbeta = batchnorm_backward(dout, cache)
#Treba očekivati da relativne greške budu između 1e-13 i 1e-8
print('dx greška: ', rel_error(dx_num, dx))
print('dgamma greška: ', rel_error(da_num, dgamma))
print('dbeta greška: ', rel_error(db_num, dbeta))

In [ ]:
np.random.seed(231)
N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
fx = lambda x: batchnorm_forward(x, gamma, beta, bn_param)[0]
fg = lambda a: batchnorm_forward(x, a, beta, bn_param)[0]
fb = lambda b: batchnorm_forward(x, gamma, b, bn_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma.copy(), dout)
db_num = eval_numerical_gradient_array(fb, beta.copy(), dout)

_, cache = batchnorm_forward(x, gamma, beta, bn_param)
dx, dgamma, dbeta = batchnorm_backward(dout, cache)
#Treba očekivati da relativne greške budu između 1e-13 i 1e-8
print('dx greška: ', rel_error(dx_num, dx))
print('dgamma greška: ', rel_error(da_num, dgamma))
print('dbeta greška: ', rel_error(db_num, dbeta))


## Batch normalizacija: alternativni prolaz unazad
Druga strategija za prolaz unazad je napisati izvode na papiru. Na primjer, izvede se jednostavna formula za sigmoid funkciju prolaza unazad pojednostavljujući gradijente na papiru.

Iznenađujuće, ispostavlja se da se na sličan način može odraditi ppojednostavljivanje za prolaz unazad batch normalizacije.  
Ulazi $X=\begin{bmatrix}x_1\\x_2\\...\\x_N\end{bmatrix}$, 
prvo računamo srednju vrijednost $\mu=\frac{1}{N}\sum_{k=1}^N x_k$ i varijansu $v=\frac{1}{N}\sum_{k=1}^N (x_k-\mu)^2.$    
Sa $\mu$ i $v$ izračunatim, možemo izračunati standardnu devijaciju $\sigma=\sqrt{v+\epsilon}$ i normalizovane podatke $Y$ sa $y_i=\frac{x_i-\mu}{\sigma}.$


Srž problema je dobiti $\frac{\partial L}{\partial X}$ za upstream gradijente $\frac{\partial L}{\partial Y}.$ Može biti izazovno za $X$ i $Y$ - dobar je pokušaj razmišljati o $x_i$ i $y_i$ prvo.

Treba izvesti $\frac{\partial L}{\partial x_i}$, oslanjajući se na Chain Rule da se prvo izračunaju među izvodi $\frac{\partial \mu}{\partial x_i}, \frac{\partial v}{\partial x_i}, \frac{\partial \sigma}{\partial x_i},$ onda sastaviti ove djeliće da se izračuna $\frac{\partial y_i}{\partial x_i}$. Treba težiti da svako od među koraka bude što je moguće jednostavniji. 

Nakon toga, implementirajte jednostavnu batch normalizaciju prolaz unazad u funkciji `batchnorm_backward_alt`. Ove dvije implementacije neće dati identične rezultate, ali bi ovaj alternativni trebao da bude brži.

In [ ]:
np.random.seed(231)
N, D = 100, 500
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

bn_param = {'mode': 'train'}
out, cache = batchnorm_forward(x, gamma, beta, bn_param)

t1 = time.time()
dx1, dgamma1, dbeta1 = batchnorm_backward(dout, cache)
t2 = time.time()
dx2, dgamma2, dbeta2 = batchnorm_backward_alt(dout, cache)
t3 = time.time()

print('dx razlika: ', rel_error(dx1, dx2))
print('dgamma razlika: ', rel_error(dgamma1, dgamma2))
print('dbeta razlika: ', rel_error(dbeta1, dbeta2))
print('speedup: %.2fx' % ((t2 - t1) / (t3 - t2)))

## FC mreže sa batch normalizacijom
Sada kada ste implementirali batch normalizaciju, vratite se na `FullyConnectedNet`  u fajlu `funkcije/fc_net.py`. Izmijenite implementaciju da dodate batch normalizaciju.

Konkretno, kada je `normalization` zastavica podešena na `"batchnorm"` u konstruktoru, trebate ubaciti batch normalizaciju ispred svake ReLU aktivacione funkcije. Izlazi iz poslednjeg sloja mreže ne treba da se noramlizuje.

In [ ]:
np.random.seed(231)
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D)
y = np.random.randint(C, size=(N,))

# Treba očekivati greške u opsegu 1e-4~1e-10 za W, 
# odnosno 1e-08~1e-10 za b,
# i 1e-08~1e-09 za beta i gammas.
for reg in [0, 3.14]:
    print('Provjera sa reg = ', reg)
    model = FullyConnectedNet([H1, H2], input_dim=D, num_classes=C,
                            reg=reg, weight_scale=5e-2, dtype=np.float64,
                            normalization='batchnorm')

    loss, grads = model.loss(X, y)
    print('Početni loss: ', loss)

    for name in sorted(grads):
        f = lambda _: model.loss(X, y)[0]
        grad_num = eval_numerical_gradient(f, model.params[name], verbose=False, h=1e-5)
        print('%s relativna greška: %.2e' % (name, rel_error(grad_num, grads[name])))
    if reg == 0: print()

# Batchnorm za duboke mreže
Pokrenuti sledeću ćeliju za treniranje šestoslojne mreže na uzorku od 1000 trening primjeraka sa i bez batch normalizacije.

In [ ]:
np.random.seed(231)

hidden_dims = [100, 100, 100, 100, 100]

num_train = 1000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

weight_scale = 2e-2
bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization='batchnorm')
model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=None)

bn_solver = Solver(bn_model, small_data,
                num_epochs=10, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-3,
                },
                verbose=True,print_every=20)
bn_solver.train()

solver = Solver(model, small_data,
                num_epochs=10, batch_size=50,
                update_rule='adam',
                optim_config={
                  'learning_rate': 1e-3,
                },
                verbose=True, print_every=20)
solver.train()

Pokretanjem sledeće ćelije će se vizuelizovati rezultati dvije mreže iznad.

In [ ]:
def plot_training_history(title, label, baseline, bn_solvers, plot_fn, bl_marker='.', bn_marker='.', labels=None):
    """pomoćna funkcija za crtanje trening istorije"""
    plt.title(title)
    plt.xlabel(label)
    bn_plots = [plot_fn(bn_solver) for bn_solver in bn_solvers]
    bl_plot = plot_fn(baseline)
    num_bn = len(bn_plots)
    for i in range(num_bn):
        label='with_norm'
        if labels is not None:
            label += str(labels[i])
        plt.plot(bn_plots[i], bn_marker, label=label)
    label='baseline'
    if labels is not None:
        label += str(labels[0])
    plt.plot(bl_plot, bl_marker, label=label)
    plt.legend(loc='lower center', ncol=num_bn+1) 

    
plt.subplot(3, 1, 1)
plot_training_history('Trening loss','Iteracija', solver, [bn_solver], \
                      lambda x: x.loss_history, bl_marker='o', bn_marker='o')
plt.subplot(3, 1, 2)
plot_training_history('Trening preciznost','Epoha', solver, [bn_solver], \
                      lambda x: x.train_acc_history, bl_marker='-o', bn_marker='-o')
plt.subplot(3, 1, 3)
plot_training_history('Validaciona preciznost','Epoha', solver, [bn_solver], \
                      lambda x: x.val_acc_history, bl_marker='-o', bn_marker='-o')

plt.gcf().set_size_inches(15, 15)
plt.show()

# Batch normalizacija i inicijalizacija
Sada ćemo napraviti mali eksperiment da proučimo interakciju između batch normalizacije i inicijalizacije težina.

Prva ćelija će trenirati osmoslojnu mrežu sa i bez batch normalizacije koristeći različite inicijalizacije težina.

In [ ]:
np.random.seed(231)

hidden_dims = [50, 50, 50, 50, 50, 50, 50]
num_train = 1000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

bn_solvers_ws = {}
solvers_ws = {}
weight_scales = np.logspace(-4, 0, num=20)
for i, weight_scale in enumerate(weight_scales):
    print('Weight scale %d / %d' % (i + 1, len(weight_scales)))
    bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization='batchnorm')
    model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=None)

    bn_solver = Solver(bn_model, small_data,
                  num_epochs=10, batch_size=50,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 1e-3,
                  },
                  verbose=False, print_every=200)
    bn_solver.train()
    bn_solvers_ws[weight_scale] = bn_solver

    solver = Solver(model, small_data,
                  num_epochs=10, batch_size=50,
                  update_rule='adam',
                  optim_config={
                    'learning_rate': 1e-3,
                  },
                  verbose=False, print_every=200)
    solver.train()
    solvers_ws[weight_scale] = solver

In [ ]:
# Crtanje rezultata eksperimenta
best_train_accs, bn_best_train_accs = [], []
best_val_accs, bn_best_val_accs = [], []
final_train_loss, bn_final_train_loss = [], []

for ws in weight_scales:
    best_train_accs.append(max(solvers_ws[ws].train_acc_history))
    bn_best_train_accs.append(max(bn_solvers_ws[ws].train_acc_history))

    best_val_accs.append(max(solvers_ws[ws].val_acc_history))
    bn_best_val_accs.append(max(bn_solvers_ws[ws].val_acc_history))

    final_train_loss.append(np.mean(solvers_ws[ws].loss_history[-100:]))
    bn_final_train_loss.append(np.mean(bn_solvers_ws[ws].loss_history[-100:]))
  
plt.subplot(3, 1, 1)
plt.title('Najbolja val preciznost vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Najbolja val preciznost')
plt.semilogx(weight_scales, best_val_accs, '-o', label='baseline')
plt.semilogx(weight_scales, bn_best_val_accs, '-o', label='batchnorm')
plt.legend(ncol=2, loc='lower right')

plt.subplot(3, 1, 2)
plt.title('Najbolja trening preciznost vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Najbolja trening preciznost')
plt.semilogx(weight_scales, best_train_accs, '-o', label='baseline')
plt.semilogx(weight_scales, bn_best_train_accs, '-o', label='batchnorm')
plt.legend()

plt.subplot(3, 1, 3)
plt.title('Krajnji trening loss vs weight initialization scale')
plt.xlabel('Weight initialization scale')
plt.ylabel('Krajnji trening loss')
plt.semilogx(weight_scales, final_train_loss, '-o', label='baseline')
plt.semilogx(weight_scales, bn_final_train_loss, '-o', label='batchnorm')
plt.legend()
plt.gca().set_ylim(1.0, 3.5)

plt.gcf().set_size_inches(15, 15)
plt.show()

# Batch normalizacija i veličina batch-a
Sada ćemo napraviti eksperiment o interakciji batch normalizacije i veličini batch-a.

Prva ćelija će trenirati šestoslojnu mrežu sa i bez batch normalizacije koristeći različite veličine batch-a.

In [ ]:
def run_batchsize_experiments(normalization_mode):
    np.random.seed(231)
    
    hidden_dims = [100, 100, 100, 100, 100]
    num_train = 1000
    small_data = {
      'X_train': data['X_train'][:num_train],
      'y_train': data['y_train'][:num_train],
      'X_val': data['X_val'],
      'y_val': data['y_val'],
    }
    n_epochs=10
    weight_scale = 2e-2
    batch_sizes = [5,10,50]
    lr = 10**(-3.5)
    solver_bsize = batch_sizes[0]

    print('Bez normalizacije: batch size = ',solver_bsize)
    model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=None)
    solver = Solver(model, small_data,
                    num_epochs=n_epochs, batch_size=solver_bsize,
                    update_rule='adam',
                    optim_config={
                      'learning_rate': lr,
                    },
                    verbose=False)
    solver.train()
    
    bn_solvers = []
    for i in range(len(batch_sizes)):
        b_size=batch_sizes[i]
        print('Normalizacija: batch size = ',b_size)
        bn_model = FullyConnectedNet(hidden_dims, weight_scale=weight_scale, normalization=normalization_mode)
        bn_solver = Solver(bn_model, small_data,
                        num_epochs=n_epochs, batch_size=b_size,
                        update_rule='adam',
                        optim_config={
                          'learning_rate': lr,
                        },
                        verbose=False)
        bn_solver.train()
        bn_solvers.append(bn_solver)
        
    return bn_solvers, solver, batch_sizes

batch_sizes = [5,10,50]
bn_solvers_bsize, solver_bsize, batch_sizes = run_batchsize_experiments('batchnorm')

In [ ]:
plt.subplot(2, 1, 1)
plot_training_history('Trening preciznost (Batch Normalization)','Epoha', solver_bsize, bn_solvers_bsize, \
                      lambda x: x.train_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)
plt.subplot(2, 1, 2)
plot_training_history('Validaciona preciznost (Batch Normalization)','Epoha', solver_bsize, bn_solvers_bsize, \
                      lambda x: x.val_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)

plt.gcf().set_size_inches(15, 10)
plt.show()

# Layer Normalizacija
Batch normalizacija je efikasna u olakšavanju treniranja mreža, ali zavisnost od veličine batch-a je čini manje korisnom u kompleksnim mrežama koje imaju ograničenja u vidu hardvera itd. 

Nekoliko alternativa je predloženo da bi se ovaj problem izbjegao. Jedna od takvih tehnika je Layer normalizacija [4]. Umjesto normalizacije po batch-u, normalizacija se vrši po karakteristikama.

[4] [Ba, Jimmy Lei, Jamie Ryan Kiros, and Geoffrey E. Hinton. "Layer Normalization." stat 1050 (2016): 21.](https://arxiv.org/pdf/1607.06450.pdf)

# Layer Normalizacija: Implementacija

Sada ćete implementirati layer normalizaciju. Ovdje ne bi trebalo da bude većih poteškoća s obzirom da je konceptualno slična batch normalizaciji.

Treba odraditi:

* U `funkcije/layers.py`, implementirati prolaz unaprijed za layer normalizaciju u funkciji `layernorm_forward`. 

* U `funkcije/layers.py`, implementirati prolaz unazad za layer normalizaciju u funkciji `layernorm_backward`. 

* Modifikovati `funkcije/fc_net.py` da bi se dodala layer normalizacija `FullyConnectedNet`. Kada `normalization` zastavica je podešena na `"layernorm"` u konstruktoru, trebate ubaciti layer normalizaciju prije svake ReLU nelinearnosti. 

In [ ]:
np.random.seed(231)
N, D1, D2, D3 =4, 50, 60, 3
X = np.random.randn(N, D1)
W1 = np.random.randn(D1, D2)
W2 = np.random.randn(D2, D3)
a = np.maximum(0, X.dot(W1)).dot(W2)

print('Prije layer normalizacije:')
print_mean_std(a,axis=1)

gamma = np.ones(D3)
beta = np.zeros(D3)
# Srednje vrijednosti bi trebalo da budu približno nula, a standardne devijacije približno jedinici
print('Posle layer normalizacije (gamma=1, beta=0)')
a_norm, _ = layernorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=1)

gamma = np.asarray([3.0,3.0,3.0])
beta = np.asarray([5.0,5.0,5.0])
# Sada srednje vrijednosti bi trebalo da budu približno beta, a standardne devijacije približno gamma
print('Posle layer normalizacije (gamma=', gamma, ', beta=', beta, ')')
a_norm, _ = layernorm_forward(a, gamma, beta, {'mode': 'train'})
print_mean_std(a_norm,axis=1)

In [ ]:
np.random.seed(231)
N, D = 4, 5
x = 5 * np.random.randn(N, D) + 12
gamma = np.random.randn(D)
beta = np.random.randn(D)
dout = np.random.randn(N, D)

ln_param = {}
fx = lambda x: layernorm_forward(x, gamma, beta, ln_param)[0]
fg = lambda a: layernorm_forward(x, a, beta, ln_param)[0]
fb = lambda b: layernorm_forward(x, gamma, b, ln_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma.copy(), dout)
db_num = eval_numerical_gradient_array(fb, beta.copy(), dout)

_, cache = layernorm_forward(x, gamma, beta, ln_param)
dx, dgamma, dbeta = layernorm_backward(dout, cache)

#Trebate očekivati relativne greške između 1e-12 i 1e-8
print('dx greška: ', rel_error(dx_num, dx))
print('dgamma greška: ', rel_error(da_num, dgamma))
print('dbeta greška: ', rel_error(db_num, dbeta))

# Layer normalizacija i veličina batch-a

Sada ćemo pokrenuti prethodni eksperiment sa veličinom batch-a sa layer normalizacijom umjesto batch normalizacije. Treba primijetiti da je manji uticaj veličine batch-a na istoriju treniranja.

In [ ]:
ln_solvers_bsize, solver_bsize, batch_sizes = run_batchsize_experiments('layernorm')

plt.subplot(2, 1, 1)
plot_training_history('Trening preciznost (Layer Normalization)','Epoha', solver_bsize, ln_solvers_bsize, \
                      lambda x: x.train_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)
plt.subplot(2, 1, 2)
plot_training_history('Validaciona preciznost (Layer Normalization)','Epoha', solver_bsize, ln_solvers_bsize, \
                      lambda x: x.val_acc_history, bl_marker='-^', bn_marker='-o', labels=batch_sizes)

plt.gcf().set_size_inches(15, 10)
plt.show()